In [1]:
import os
import time
import threading
import multiprocessing

In [2]:
NUM_WORKERS = 4

In [3]:
def only_sleep():
    '''Do nothing, wait for a timer to expire'''
    print('PID: %s, Process Name: %s, Thread Name: %s' % (os.getpid(), multiprocessing.current_process().name, threading.current_thread().name))
    time.sleep(1)
 
 
def crunch_numbers():
    '''Do some computations'''
    print('PID: %s, Process Name: %s, Thread Name: %s' % (os.getpid(),multiprocessing.current_process().name,threading.current_thread().name))
    x = 0
    while x < 10000000:
        x += 1

In [4]:
## Run tasks serially
start_time = time.time()
for _ in range(NUM_WORKERS):
    only_sleep()
end_time = time.time()
 
print("Serial time=", end_time - start_time)

PID: 14436, Process Name: MainProcess, Thread Name: MainThread
PID: 14436, Process Name: MainProcess, Thread Name: MainThread
PID: 14436, Process Name: MainProcess, Thread Name: MainThread
PID: 14436, Process Name: MainProcess, Thread Name: MainThread
Serial time= 4.033903121948242


In [5]:
# Run tasks using threads
start_time = time.time()
threads = [threading.Thread(target=only_sleep) for _ in range(NUM_WORKERS)]
[thread.start() for thread in threads]
[thread.join() for thread in threads]
end_time = time.time()
print("Threads time=", end_time - start_time)

PID: 14436, Process Name: MainProcess, Thread Name: Thread-6
PID: 14436, Process Name: MainProcess, Thread Name: Thread-7
PID: 14436, Process Name: MainProcess, Thread Name: Thread-8
PID: 14436, Process Name: MainProcess, Thread Name: Thread-9
Threads time= 1.0295541286468506


In [6]:
# Run tasks using processes
start_time = time.time()
processes = [multiprocessing.Process(target=only_sleep) for _ in range(NUM_WORKERS)]
[process.start() for process in processes]
[process.join() for process in processes]
end_time = time.time()
 
print("Parallel time=", end_time - start_time)

Parallel time= 0.2070159912109375


In [7]:
start_time = time.time()
for _ in range(NUM_WORKERS):
    crunch_numbers()
end_time = time.time()
 
print("Serial time=", end_time - start_time)

PID: 14436, Process Name: MainProcess, Thread Name: MainThread
PID: 14436, Process Name: MainProcess, Thread Name: MainThread
PID: 14436, Process Name: MainProcess, Thread Name: MainThread
PID: 14436, Process Name: MainProcess, Thread Name: MainThread
Serial time= 4.3767311573028564


In [8]:
start_time = time.time()
threads = [threading.Thread(target=crunch_numbers) for _ in range(NUM_WORKERS)]
[thread.start() for thread in threads]
[thread.join() for thread in threads]
end_time = time.time()
 
print("Threads time=", end_time - start_time)

PID: 14436, Process Name: MainProcess, Thread Name: Thread-10
PID: 14436, Process Name: MainProcess, Thread Name: Thread-11
PID: 14436, Process Name: MainProcess, Thread Name: Thread-12
PID: 14436, Process Name: MainProcess, Thread Name: Thread-13
Threads time= 5.345181226730347


In [9]:
start_time = time.time()
processes = [multiprocessing.Process(target=crunch_numbers) for _ in range(NUM_WORKERS)]
[process.start() for process in processes]
[process.join() for process in processes]
end_time = time.time()
 
print("Parallel time=", end_time - start_time)

Parallel time= 0.21001338958740234


In [10]:
%%time
ls = []
def count(n):
    for i in range(1, n+1):
        ls.append(i)
        time.sleep(0.01)

def count2(n):
    for i in range(1, n+1):
        ls.append(i)
        time.sleep(0.02)
        

x = threading.Thread(target=count, args=(10,))
x.start()


y = threading.Thread(target=count2, args=(10,))
y.start()

x.join()
y.join()

print(ls)

[1, 1, 2, 2, 3, 4, 3, 5, 4, 6, 7, 8, 5, 9, 10, 6, 7, 8, 9, 10]
Wall time: 298 ms


In [11]:
import asyncio
loop = asyncio.get_event_loop()

In [12]:
%%time
ls = []

async def count():
    for i in range(1, 10):
        ls.append(i)
        
        await asyncio.sleep(0.01)

async def count2():
    for i in range(1, 10):
        ls.append(i)
       
        await asyncio.sleep(0.01)

#asyncio.gather(count(10), count2(10))

# loop.create_task(count(10))
# loop.create_task(count2(10))

async def main():
    await asyncio.gather(count(), count2())

Wall time: 0 ns


In [13]:
await main()
print(ls)

[1, 1, 2, 2, 3, 3, 4, 4, 5, 5, 6, 6, 7, 7, 8, 8, 9, 9]
